import dan inisialisasi library dan variable

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from dotenv import load_dotenv
import os

c:\Users\Revian\Documents\Alixzer\local_system\tele_bot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
dataset_path = os.getenv('DATASET_PATH')
model_path = os.getenv('MODEL_PATH')

load dataset dan melihat dataset

In [3]:
# Load the entire dataset
data_full = pd.read_csv(dataset_path, sep='\t', header=None, names=['text', 'sentiment'])

# Display the first few rows to confirm loading
data_full.head()

,text,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


proses awal

In [4]:
label_dict = {'positive': 1, 'neutral': 0, 'negative': 2}
data_full['sentiment'] = data_full['sentiment'].replace(label_dict)

C:\Users\Revian\AppData\Local\Temp\ipykernel_30668\2928587345.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_full['sentiment'] = data_full['sentiment'].replace(label_dict)


In [5]:
tokenizer = AutoTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
#Tokenisasi
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

membagi dataset menjadi training dan validasi

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_full['text'], data_full['sentiment'], test_size=0.1)

train_dataset = Dataset.from_pandas(pd.DataFrame({'text': train_texts, 'labels': train_labels}))
val_dataset = Dataset.from_pandas(pd.DataFrame({'text': val_texts, 'labels': val_labels}))

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

Map: 100%|██████████| 1100/1100 [00:00<00:00, 18625.40 examples/s]


training

In [7]:
#menghitung metrik akurasi
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [8]:
model = AutoModelForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Inisialisasi Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

c:\Users\Revian\Documents\Alixzer\local_system\tele_bot\.venv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Mulai training
trainer.train()

# Evaluasi
trainer.evaluate()

In [ ]:
# Menyimpan model dan tokenizer yang telah fine-tune
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)